In [2]:
import numpy as np
import pandas as pd
from prophet import Prophet
import tqdm

# 1. Load and preprocess data

In [3]:
cycling_urls = [
    "https://opendata.muenchen.de/dataset/022a11ff-4dcb-4f03-b7dd-a6c94a094587/resource/66be7619-a672-4382-bf88-e3688c5abc2b/download/rad_2008_15min_06_06_23_r.csv",
    "https://opendata.muenchen.de/dataset/022a11ff-4dcb-4f03-b7dd-a6c94a094587/resource/3ef8aad9-a6b0-4c97-a6b7-8c3a63226b37/download/rad_2009_15min_06_06_23_r.csv",
    "https://opendata.muenchen.de/dataset/022a11ff-4dcb-4f03-b7dd-a6c94a094587/resource/6558a5f9-2c96-4e4b-985d-8eb99b7b73b1/download/rad_2010_15min_06_06_23_r.csv",
    "https://opendata.muenchen.de/dataset/022a11ff-4dcb-4f03-b7dd-a6c94a094587/resource/86962013-4854-4deb-aaf9-36e3770cde24/download/rad_2011_15min_export_06_06_23_r.csv",
    "https://opendata.muenchen.de/dataset/022a11ff-4dcb-4f03-b7dd-a6c94a094587/resource/ff5d2ebf-dde6-4f21-9c68-2aab74addeec/download/rad_2012_15min_06_06_23_r.csv",
    "https://opendata.muenchen.de/dataset/022a11ff-4dcb-4f03-b7dd-a6c94a094587/resource/9f4f798f-0ad1-4e86-8157-15c5e46eaf91/download/rad_2013_15min_06_06_23_r.csv",
    "https://opendata.muenchen.de/dataset/022a11ff-4dcb-4f03-b7dd-a6c94a094587/resource/784b925b-1d5f-43d3-8353-fd5d02fc7c53/download/rad_2014_15min_06_06_23_r.csv",
    "https://opendata.muenchen.de/dataset/022a11ff-4dcb-4f03-b7dd-a6c94a094587/resource/d6b3a72c-b180-40a0-a2ab-d97040737f20/download/rad_2015_15min_06_06_23_r.csv",
    "https://opendata.muenchen.de/dataset/022a11ff-4dcb-4f03-b7dd-a6c94a094587/resource/3913d9e6-1be8-4ee4-ab88-1266cbf161f1/download/rad_2016_15min_06_06_23_r.csv",
    "https://opendata.muenchen.de/dataset/022a11ff-4dcb-4f03-b7dd-a6c94a094587/resource/694b9927-b4d5-4e8f-9c62-09b8ac03c39a/download/rad_2017_15min_06_06_23_r.csv",
    "https://opendata.muenchen.de/dataset/022a11ff-4dcb-4f03-b7dd-a6c94a094587/resource/0a97a624-daa4-4cd8-a820-7d2fa6ffe89a/download/rad_2018_15min_06_06_23_r.csv",
    "https://opendata.muenchen.de/dataset/022a11ff-4dcb-4f03-b7dd-a6c94a094587/resource/893e1f16-6504-4f4f-b8b3-f907ef406cd5/download/rad_2019_15min_06_06_23_r.csv",
    "https://opendata.muenchen.de/dataset/022a11ff-4dcb-4f03-b7dd-a6c94a094587/resource/9be77b23-a444-4ba5-be9e-7f8594aa0188/download/rad_2020_15min_06_06_23_r.csv",
    "https://opendata.muenchen.de/dataset/022a11ff-4dcb-4f03-b7dd-a6c94a094587/resource/205c5c9e-9689-4c28-97cb-e575c6c772ce/download/rad_2021_15min_06_06_23_r.csv",
    "https://opendata.muenchen.de/dataset/022a11ff-4dcb-4f03-b7dd-a6c94a094587/resource/e65e0e26-ce1f-4e58-9260-beccac196e75/download/rad_2022_15min_06_06_23_r.csv"
]
df_cycling = pd.concat([pd.read_csv(url) for url in cycling_urls])

/var/folders/3g/fyfxfrj93vdg3cbm5kkhp1xh0000gn/T/ipykernel_17069/790155708.py:18: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cycling = pd.concat([pd.read_csv(url) for url in cycling_urls])
/var/folders/3g/fyfxfrj93vdg3cbm5kkhp1xh0000gn/T/ipykernel_17069/790155708.py:18: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cycling = pd.concat([pd.read_csv(url) for url in cycling_urls])
/var/folders/3g/fyfxfrj93vdg3cbm5kkhp1xh0000gn/T/ipykernel_17069/790155708.py:18: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cycling = pd.concat([pd.read_csv(url) for url in cycling_urls])
/var/folders/3g/fyfxfrj93vdg3cbm5kkhp1xh0000gn/T/ipykernel_17069/790155708.py:18: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cycling = pd.concat([pd.read_csv(url) for url in cycling_url

In [8]:
# temp = df_cycling

In [9]:
cols = ["datum", "uhrzeit_start", "zaehlstelle", "richtung_1", "richtung_2", "kommentar"]
df_cycling = df_cycling[cols]

df_cycling = df_cycling.rename(columns={
    'datum': 'date',
    'uhrzeit_start': 'time_start',
    'zaehlstelle': 'station',
    'richtung_1': 'direction_1',
    'richtung_2': 'direction_2',
    'kommentar': 'comment'
})

df_cycling['ds'] = df_cycling['date'] + ' ' + df_cycling['time_start'].astype(str)

df_cycling['date'] = pd.to_datetime(df_cycling['date'])
df_cycling['time_start'] = pd.to_datetime(df_cycling['time_start'])

df_cycling['year'] = df_cycling['date'].dt.year
df_cycling['month'] = df_cycling['date'].dt.month
df_cycling['day'] = df_cycling['date'].dt.day
df_cycling['dayofweek'] = df_cycling['date'].dt.dayofweek  # Monday=0, Sunday=6
df_cycling['hour'] = df_cycling['time_start'].dt.hour
df_cycling = df_cycling.drop(columns=["time_start"])

df_cycling['hour_weekday'] = df_cycling['dayofweek'] * 24 + df_cycling['hour']
df_cycling = df_cycling[(df_cycling.direction_1 != -1) & (df_cycling.direction_2 != -1)]

/var/folders/3g/fyfxfrj93vdg3cbm5kkhp1xh0000gn/T/ipykernel_17069/3872561769.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_cycling['time_start'] = pd.to_datetime(df_cycling['time_start'])


In [19]:
# Define mappings for comments
comment_mapping = {
    "Zählstelle noch nicht in Betrieb": "station not yet in operation",
    "Radweg vereist / nach Schneefall nicht geräumt / keine Messung möglich": "bikelane icy / not cleared after snowfall / no measure possible",
    "Baustelle": "construction",
    "Austausch Sensor": "replacing sensor",
    "Ausfall nach Beschädigung": "failure after damage"
}

# Replace comments using the defined mapping
df_cycling['comment'] = df_cycling['comment'].map(comment_mapping).fillna(df_cycling['comment'])

# Just for checking
print(df_cycling[df_cycling['comment'].notna()].direction_1.unique()) # NaN
print(df_cycling[df_cycling['comment'].notna()].direction_2.unique()) # NaN



In [71]:
# df_cycling.to_csv('/Users/juntaekwon/work_r/Consulting Project/data_folder/df_cycling.csv', index=False, compression='gzip')
# Note: df_cycling is now in data_folder
# We don't have to do loading and preprocessing data agian but just load from the folder
df_cycling = pd.read_csv('/Users/juntaekwon/work_r/Consulting Project/data_folder/df_cycling.csv', compression='gzip')


In [77]:
a = pd.read_csv('/Users/juntaekwon/work_r/Consulting Project/data_folder/df_cycling.csv', compression='gzip')

/var/folders/3g/fyfxfrj93vdg3cbm5kkhp1xh0000gn/T/ipykernel_17069/907981283.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  a = pd.read_csv('/Users/juntaekwon/work_r/Consulting Project/data_folder/df_cycling.csv', compression='gzip')


array([nan, 'station not yet in operation',
       'bikelane icy / not cleared after snowfall / no measure possible',
       'construction', 'replacing sensor', 'failure after damage'],
      dtype=object)